In [1]:
import os, fnmatch
import io
import csv
from read_data import convert_pdf_to_txt
from PIL import Image
import pytesseract
from wand.image import Image as wi
import hashlib
import pandas as pd
import numpy as np
import pprint as pp
from urllib.request import urlopen
import time
from datetime import datetime

In [2]:
def get_size(st):
    if os.path.isfile(st):
        return os.path.getsize(st)

In [3]:
def conv_pdf(pdf):
    if os.path.isfile(pdf):
        print('--> ' + str(datetime.now()) + ' ' + pdf)
        with open(pdf, 'rb') as f:
            # text from pdf
            bfr = io.BufferedReader(f)
            pdf_text = convert_pdf_to_txt(bfr)
            
            # recognized text (OCR) from pdf
            if len(pdf_text.strip()) <= 50:
                pdf_file = wi(filename=pdf, resolution=200)
                pdfImage = pdf_file.convert('jpeg')
                imageBlobs = []
                for img in pdfImage.sequence:
                    imgPage = wi(image=img)
                    imageBlobs.append(imgPage.make_blob('jpeg'))

                recognized_text = []

                for imgBlob in imageBlobs:
                    im = Image.open(io.BytesIO(imgBlob))
                    text = pytesseract.image_to_string(im, lang = 'eng')
                    recognized_text.append(text)

                recognized_text = '\n\n\n'.join(recognized_text)

            pdf_text = pdf_text if len(pdf_text.strip()) > 50 else recognized_text
            
            return pdf_text

In [4]:
def get_filename_by_url(url):
    file_id = url.split('=')[-1]
    
    def find(pattern):
        result = []
        for root, dirs, files in os.walk('./pdfs_2018/'):
            for name in files:
                if fnmatch.fnmatch(name, pattern):
                    result.append(os.path.join(root, name))
        return result
    
    f_arr = find('*' + file_id + '.pdf')

    if len(f_arr) == 1:
        return f_arr[0]
    elif len(f_arr) > 1:
        raise ValueError('More than one file exist')
    elif len(f_arr) == 0:
        raise ValueError('No file')

In [5]:
maine_csv = pd.read_csv('maine_2018_new.csv', index_col = 0)

In [6]:
maine_csv.describe(include='all')

,chamber,date,html,md5,session,state,text,topic,url,size,filename
count,726,726,0.0,726,726,726,0.0,726,726,7.260000e+02,726
unique,2,64,NaN,723,1,1,NaN,1,726,NaN,726
top,House,Tue Feb 13 13:00:00 EST 2018,NaN,09b63ec63527a75ae62c6b169b169e2f,128th,maine,NaN,#TODO,https://legislature.maine.gov/legis/bills/getT...,NaN,"./pdfs_2018/Hymanson, Patty - 78747.pdf"
freq,490,71,NaN,3,726,726,NaN,726,1,NaN,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.419693e+05,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.932919e+05,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.240000e+04,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.972650e+04,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.243650e+04,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.291382e+05,NaN


In [7]:
maine_csv = maine_csv[600:]

In [8]:
maine_csv.describe(include='all')

,chamber,date,html,md5,session,state,text,topic,url,size,filename
count,126,126,0.0,126,126,126,0.0,126,126,1.260000e+02,126
unique,2,44,NaN,126,1,1,NaN,1,126,NaN,126
top,House,Tue Feb 13 13:00:00 EST 2018,NaN,b744ec6010cf9e3bf1feb2c561585842,128th,maine,NaN,#TODO,https://legislature.maine.gov/legis/bills/getT...,NaN,"./pdfs_2018/Meyers, Daniel - 82095.pdf"
freq,95,15,NaN,1,126,126,NaN,126,1,NaN,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.029778e+05,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.418662e+05,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.498780e+05,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.826505e+05,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.206020e+05,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.434608e+05,NaN


In [9]:
maine_csv['text'] = maine_csv.filename.apply(conv_pdf)

--> 2018-02-17 02:48:01.644656 ./pdfs_2018/Fortin, Mark - 58879.pdf
--> 2018-02-17 02:48:03.594390 ./pdfs_2018/Cook, Susan - 84313.pdf
--> 2018-02-17 02:48:05.026673 ./pdfs_2018/Doran, Dana - 82135.pdf
--> 2018-02-17 02:48:06.882540 ./pdfs_2018/Dickstein, Carla - 85376.pdf
--> 2018-02-17 02:48:08.885635 ./pdfs_2018/McGinley, Heidi - 69695.pdf
--> 2018-02-17 02:48:10.708021 ./pdfs_2018/Tuell, William - 58867.pdf
--> 2018-02-17 02:48:12.855372 ./pdfs_2018/Lister, Hillary - 85141.pdf
--> 2018-02-17 02:48:14.704331 ./pdfs_2018/Agren, Eric - 84436.pdf
--> 2018-02-17 02:48:16.770927 ./pdfs_2018/Whitney, Brian - 80166.pdf
--> 2018-02-17 02:49:17.805554 ./pdfs_2018/Dion, Mark - 82987.pdf
--> 2018-02-17 02:49:20.244998 ./pdfs_2018/Stinneford, Eric - 85136.pdf
--> 2018-02-17 02:49:23.060199 ./pdfs_2018/Gerrish, Karen - 84629.pdf
--> 2018-02-17 02:49:25.901275 ./pdfs_2018/Abello, Tom - 82921.pdf
--> 2018-02-17 02:49:27.756896 ./pdfs_2018/Strauch, Patrick - 82930.pdf
--> 2018-02-17 02:49:28.941974

--> 2018-02-17 03:01:46.977422 ./pdfs_2018/Morgan, Claude - 84504.pdf
--> 2018-02-17 03:02:00.339389 ./pdfs_2018/Gates, Russell - 82912.pdf
--> 2018-02-17 03:04:20.166140 ./pdfs_2018/Voorhees, Dylan - 81032.pdf
--> 2018-02-17 03:04:27.047934 ./pdfs_2018/Bennett, Nick - 85142.pdf
--> 2018-02-17 03:04:52.080872 ./pdfs_2018/Laberge, Paul - 85137.pdf
--> 2018-02-17 03:05:13.021717 ./pdfs_2018/Bennett, Nick - 82920.pdf
--> 2018-02-17 03:05:43.882263 ./pdfs_2018/Libby, Nathan - 84621.pdf
--> 2018-02-17 03:07:05.716276 ./pdfs_2018/Hayes, Terry - 72543.pdf
--> 2018-02-17 03:07:54.469054 ./pdfs_2018/Champion, Milton - 85215.pdf


In [10]:
maine_csv

,chamber,date,html,md5,session,state,text,topic,url,size,filename
bill_name,,,,,,,,,,,
"LD 861, HP 610",House,Fri Mar 31 09:00:00 EDT 2017,NaN,f6a0807853c3e7d3c3bd7c6d259222a8,128th,maine,      \n\n3\n§\n\n¢ \n\n<1\n\ns \n\nCriminal J...,#TODO,https://legislature.maine.gov/legis/bills/getT...,149878,"./pdfs_2018/Fortin, Mark - 58879.pdf"
"LD 1781, HP 1227",House,Tue Jan 30 13:00:00 EST 2018,NaN,3e8c9c90ba9412a6004633c8a999bbd3,128th,maine,            \n\nI \n\nbiw \n\nTESTIMONY IN OPP...,#TODO,https://legislature.maine.gov/legis/bills/getT...,150015,"./pdfs_2018/Cook, Susan - 84313.pdf"
"LD 1754, SP 653",House & Senate,Thu Jan 18 13:00:00 EST 2018,NaN,6741f41005556aca631e9e5e60b970e7,128th,maine,     \n \n\nTestimony of \nDANA A. DORAN \nExe...,#TODO,https://legislature.maine.gov/legis/bills/getT...,150161,"./pdfs_2018/Doran, Dana - 82135.pdf"
"LD 1774, HP 1223",House,Wed Feb 14 15:00:00 EST 2018,NaN,5d5e8707b2a625379ffa85ea4e83018d,128th,maine,             \n                               ...,#TODO,https://legislature.maine.gov/legis/bills/getT...,151401,"./pdfs_2018/Dickstein, Carla - 85376.pdf"
"LD 228, SP 74",House & Senate,Thu Apr 20 13:00:00 EDT 2017,NaN,905468f4f2e3b69671cfa867bfe9c722,128th,maine,Ell’\n\nC \n\nl\/lCLA's mission is to support ...,#TODO,https://legislature.maine.gov/legis/bills/getT...,151756,"./pdfs_2018/McGinley, Heidi - 69695.pdf"
"LD 861, HP 610",House,Fri Mar 31 09:00:00 EDT 2017,NaN,19c385f1c5325f11cd256f5cef668de5,128th,maine,t\n\nl\n\nl\n\nl\n\nl \n\nl\n\n \n \n      \n\...,#TODO,https://legislature.maine.gov/legis/bills/getT...,152268,"./pdfs_2018/Tuell, William - 58867.pdf"
"LD 1797, HP 1242",House,Wed Feb 14 10:00:00 EST 2018,NaN,c8fb4a356e9bd9ae739c8a30cf0cf2c7,128th,maine,                 \n \n     \n\n>\n2\n\nx \n\n‘...,#TODO,https://legislature.maine.gov/legis/bills/getT...,153613,"./pdfs_2018/Lister, Hillary - 85141.pdf"
"LD 1772, SP 659",House & Senate,Tue Jan 30 14:00:00 EST 2018,NaN,d33ecc69ae4b784093b9390d1f6fe785,128th,maine,"Senator Amy Volk, Chair \nRepresentative Ryan ...",#TODO,https://legislature.maine.gov/legis/bills/getT...,153763,"./pdfs_2018/Agren, Eric - 84436.pdf"
"LD 1613, SP 568",House & Senate,Mon Jul 17 10:00:00 EDT 2017,NaN,e85436e45616a0016277b32c45e907f1,128th,maine,Mame Technoirng Ersfitute\n\nTESTHVIONY OF BRI...,#TODO,https://legislature.maine.gov/legis/bills/getT...,155491,"./pdfs_2018/Whitney, Brian - 80166.pdf"


In [11]:
maine_csv.loc[:, ['chamber','date','html','md5','session','state','text','topic','filename']].to_csv('maine-2018-pt4.csv')

In [ ]:
maine_csv['size'] = maine_csv.url.apply(get_filename_by_url).apply(get_size)
maine_csv['filename'] = maine_csv.url.apply(get_filename_by_url)
maine_csv = maine_csv.sort_values('size')